In [1]:
import os,sys,progressbar
import numpy as np
import pandas as pd
import time
from IPython.display import clear_output
sys.path.insert(1, '/home/amarins/ComponentSeparation/gmca/scripts')
import Extension4BINGO as cs
import copy

In [2]:
#######################################################
################   GENERAL INFORMATIONS   #############
#######################################################
n_realizations   = 3 #If you want to use all realizations, please, put -1
method           = "ICA"
wtransform       = ["identity"]
maps_wout_mean   = True
apply_mask       = False
add_noise        = False

#######################################################
################   WAVELETS PARAMETERS   ##############
#######################################################
#####
# The main aim is to have:
# Starlet, Axisymmetric, spin-Directional, Wavelets Standard from PyWavelets, Curvelets, Counturlets, Shearlets, Ridgelets and so on
#####
J     = 1  #number of scales
use_c = True  # if you will use wavelet scale in the analysis

##############
# S2LET code
# If you to use wtransforms by S2Let code, please, fill in the variables below:
L        = -1 #If you write L<0, it will use L=3*nside
J_min    = 1
B        = 5     
N        = 3  # Number of directions (This is for Directional only)
spin     = 0  # set to 0 for temperature. if non-zero, plotting routines must be changed! (This is for Directional only)
upsample = 0  # 1 means all scales at full resolution L # 0 means multiresolution wavelet transform (This is for Directional only)
# In the S2LET code, J scales is defined by code and not by J above.

##############
# PyWavelets
Jpwt     = 1 #number of scales
pywttype = "haar" 

##############
# Needlets
needlet   = "mexican" # either mexican(=gaussian), or standard
Bneed     = 1.15            # filter is defined by function b(l/B**j), where j=freq
p         = 0.9             # this value is only for mexican needlet
fneed     = [15,24,30,34]   # number of freqs will be the number of scales (wavelet maps). Center of i-band will be approached given by l_center_i = B**freq_i (p=0)
lmax_need = -1

##############
# Curvelets
##############
# Counturlets
##############
# Shearlets
##############
# Ridgelets

#######################################################
#### COMPONENT SEPARATION #############################
#######################################################
n_s = 1  #number of sources to be estimated
#Warning! if this one is 1, it's impossible build projection maps

######## FastICA PARAMETERS 
whiten = True  ######## Maintain True
fun = 'logcosh' #exp,logcosh or
max_iter = 20
tol = 0.01

######## GMCA PARAMETERS   
mints = 0.05 # min threshold (what is sparse compared to noise?)
nmax  = 100 # number of iterations (usually 100 is safe)
L0    = 0   # switch between L0 norm (1) or L1 norm (0)

#######################################################
AInit     = None
ColFixed  = None
whitening = False
epsi      = 1e-3
verbose   = False
#GMCAExtension
div          = 1 #  J+1  #J/div will should be even number
without_covx = True # if your mixmatrix estimated will use covariance matrix of the observer data with ponderation
# 0 <= noise factor <= 1
#noise_factor = 1  <-------------- Preciso que os mapas sejam todos separados

#######################################################
################   DEBIAS PARAMETERS   ################
#######################################################
seed_used = 10     #If False, it will be chosen a random realization. Otherwise, it is necessary to write which realization that will be used. For example, if you want to use L10, it is necessary you to write 10 (it's string). Also, if you chose realizaton out of set, it will be chosen a random value.

#######################################################
################   PATHS PARAMETERS   #################
#######################################################
#path outputs
pathout       = "/home/amarins/ComponentSeparation/gmca/outputs/Gtest_addCommand" #Put here your path to the output cls
cl_type_save  = "reconstruction" #You should choice between reconstruction or residuals cls values
savefits      = True# or False
#######################################################
################   NAME FILES PARAMETERS   ############
#######################################################
# Name of FITS files inside of the pathmaps
name_mask = "Mask_Bin.fits" #put this file in the same directory of the other maps

#Directory names
dir_observed  = "/media/new-drive/CS_Cubes/White_Noise/input_wn"
dir_mask      = "/media/new-drive/amarins/maps/mask"
dir_prior     = "/media/new-drive/CS_Cubes/White_Noise/prior_wn"
dir_noise     = "/media/new-drive/CS_Cubes/Only_White_Noise_Masked"
dir_pure      = "/media/new-drive/CS_Cubes/No_Noise/prior_nn"
dir_projprior = "/media/new-drive/CS_Cubes/White_Noise/prior_wn"
dir_projnoise = "/media/new-drive/CS_Cubes/Only_White_Noise_Masked"
dir_projpure  = "/media/new-drive/Cs_cubes/No_Noise/prior_nn"

#######################################################
restart = True
#if you were running the code and it broke, and you used "n_realizations" <total, the next run will assume restart = True, because it's impossible to know which nseed you used
#warning: if you don't have the directories and put "restart=False" code wont be work. Please, put "True" to create directories and to run.
#You cant create "proj" directories if you dont have all others directories

In [3]:
params_maps = pd.Series({"without_mean":maps_wout_mean, "apply_mask":apply_mask, "add_noise":add_noise, "cl_type_save":cl_type_save})
params_CS   = pd.Series({"restart":restart,"n_realizations":n_realizations, "method":method,
                         "A_ini":AInit, "ColFixed":ColFixed, "whitening":whitening, "epsi":epsi, "verbose":verbose, "ns":n_s, "mints":mints,"nmax":nmax, "L0":L0, "division":div, "without_covx":without_covx, "seed_used":seed_used,
                         "whiten":whiten, "fun":fun, "max_iter":max_iter, "tol":tol})
params_WT   = pd.Series({"wtransform":np.asarray(wtransform), "use_c":use_c, "J":J, 
                         "L":L, "Jmin":J_min, "B": B, "N":N, "spin":spin, "upsample":upsample,
                         "Jpwt":Jpwt, "pywttype":pywttype.lower(),
                         "needlet":needlet, "Bneed":Bneed, "p":p, "fneed": fneed, "lmax_need":lmax_need})
params_path = pd.Series({"pathout":pathout, "dir_observed":dir_observed, "dir_mask":dir_mask, "dir_noise":dir_noise, "dir_prior":dir_prior,"dir_pure":dir_pure, "name_mask":name_mask})

### Foregrounds + HI + Noise
$C_{\ell}^{\textrm{FG+HI+N}}$

In [4]:
names, nseed0 = cs.nsamples(params_CS,params_path,path=params_path.dir_observed, listdir="21cm")
params_maps["getdata"] = "observed"
subdirs = cs.checkdir(params_path.pathout, subdirs=["21cm","foregrounds","mixmatrix"],restart=params_CS["restart"])

bar = progressbar.ProgressBar(maxval=names.size)
for i,iname in enumerate(names):
    clear_output(wait=True)
    bar.update(i)
    params_cs, params_wt = cs.load(params_CS,params_WT)
    params_path["name_observed"] = iname
    X = cs.getmaps(params_maps, params_path)
    X = cs.adaptation_maps(X, params_maps, params_path)
    X = cs.maps2CSmaps(X, params_wt, params_cs)
    params_maps["iseed"]="L"+str(nseed0[i])
    cs.saveouts(mrec=X, params_path=params_path, params_maps=params_maps, params_WT=params_wt, params_CS=params_cs, subdirs=subdirs)
    #del X
clear_output(wait=True)
bar.finish()

100% (3 of 3) |##########################| Elapsed Time: 0:08:12 Time:  0:08:12


### Noise
$C_{\ell}^{\textrm{N}}$

In [5]:
params_maps["getdata"] = "noise"
subdirs                = ["noise"]
if not os.path.isdir(os.path.join(params_path.pathout,subdirs[0])):
    os.makedirs(os.path.join(params_path.pathout,subdirs[0]))

names, nseed           = cs.nsamples(params_CS,params_path,path=params_path.dir_noise, listdir=params_maps["getdata"],nseed_used=nseed0)

bar = progressbar.ProgressBar(maxval=names.size)
for i,iname in enumerate(names):
    clear_output(wait=True)
    bar.update(i)
    params_cs, params_wt = cs.load(params_CS,params_WT)
    params_path["name_noise"] = iname
    X = cs.getmaps(params_maps, params_path)
    X = cs.adaptation_maps(X, params_maps, params_path)
    params_maps["iseed"]="L"+str(nseed[i])
    cs.saveouts(mrec=X, params_path=params_path, params_maps=params_maps, params_WT=params_wt, params_CS=params_cs, subdirs=subdirs)    

bar.finish()

100% (3 of 3) |##########################| Elapsed Time: 0:04:31 Time:  0:04:31


### Prior
$C_{\ell}^{\textrm{prior}}$

In [6]:
params_maps["getdata"] = "prior"
subdirs = ["prior"]
if not os.path.isdir(os.path.join(params_path.pathout,subdirs[0])):
    os.makedirs(os.path.join(params_path.pathout,subdirs[0]))

names, nseed = cs.nsamples(params_CS,params_path,path=params_path.dir_prior, listdir=params_maps["getdata"],nseed_used=nseed0)

bar = progressbar.ProgressBar(maxval=names.size)    
for i,iname in enumerate(names):
    clear_output(wait=True)
    bar.update(i) 
    params_cs, params_wt = cs.load(params_CS,params_WT)
    params_path["name_prior"] = iname
    X = cs.getmaps(params_maps, params_path)
    X = cs.adaptation_maps(X, params_maps, params_path)
    params_maps["iseed"]="L"+str(nseed[i])
    cs.saveouts(mrec=X, params_path=params_path, params_maps=params_maps, params_WT=params_wt, params_CS=params_cs, subdirs=subdirs)    
bar.finish()

100% (3 of 3) |##########################| Elapsed Time: 0:03:57 Time:  0:03:57


### 21cm pure
$C_{\ell}^{\textrm{pure}}$

In [7]:
params_maps["getdata"] = "pure"
subdirs = ["pure"]
if not os.path.isdir(os.path.join(params_path.pathout,subdirs[0])):
    os.makedirs(os.path.join(params_path.pathout,subdirs[0]))

names, nseed = cs.nsamples(params_CS,params_path,path=params_path.dir_pure, listdir=params_maps["getdata"],nseed_used=nseed0)
    
bar = progressbar.ProgressBar(maxval=names.size)
for i,iname in enumerate(names):
    clear_output(wait=True)
    bar.update(i)    
    time0 = time.time()
    params_cs, params_wt = cs.load(params_CS,params_WT)
    params_path["name_pure"] = iname
    X = cs.getmaps(params_maps, params_path)
    X = cs.adaptation_maps(X, params_maps, params_path)
    params_maps["iseed"]="L"+str(nseed[i])
    cs.saveouts(mrec=X, params_path=params_path, params_maps=params_maps, params_WT=params_wt, params_CS=params_cs, subdirs=subdirs)    
    
bar.finish()

100% (3 of 3) |##########################| Elapsed Time: 0:03:48 Time:  0:03:48


### 21cm proj pure
$C_{\ell}^{\textrm{proj pure}}$

In [8]:
params_maps["getdata"] = "pure"
subdirs = ["projpure"]
if not os.path.isdir(os.path.join(params_path.pathout,subdirs[0])):
    os.makedirs(os.path.join(params_path.pathout,subdirs[0]))

names, nseed = cs.nsamples(params_CS,params_path,path=params_path.dir_pure, listdir="proj"+params_maps["getdata"],nseed_used=nseed0)

if nseed0[nseed0==params_CS.seed_used].size>0:
    L0 = "L{}".format(params_CS.seed_used)
else:
    L0 = "L{}".format(np.random.choice(nseed0,size=1)[0])

A   = cs.loadmixmatrix(params_path.pathout,"mixmatrix")
bar = progressbar.ProgressBar(maxval=names.size)
for i,iname in enumerate(names):
    clear_output(wait=True)
    bar.update(i)    
    params_cs, params_wt = cs.load(params_CS,params_WT)
    params_path["name_pure"] = iname
    X = cs.getmaps(params_maps, params_path)
    X = cs.adaptation_maps(X, params_maps, params_path)
    params_maps["iseed"]="L"+str(nseed[i])
    cs.saveouts(mrec=X, A=A[L0], params_path=params_path, params_maps=params_maps, params_WT=params_wt, params_CS=params_cs, subdirs=subdirs)    


bar.finish()

100% (3 of 3) |##########################| Elapsed Time: 0:03:55 Time:  0:03:55


### 21cm  proj noise
$C_{\ell}^{\textrm{proj noise}}$

In [9]:
params_maps["getdata"] = "noise"
subdirs = ["projnoise"]
if not os.path.isdir(os.path.join(params_path.pathout,subdirs[0])):
    os.makedirs(os.path.join(params_path.pathout,subdirs[0]))

names, nseed = cs.nsamples(params_CS,params_path,path=params_path.dir_noise, listdir="proj"+params_maps["getdata"],nseed_used=nseed0)

if nseed0[nseed0==params_CS.seed_used].size>0:
    L0 = "L{}".format(params_CS.seed_used)
else:
    L0 = "L{}".format(np.random.choice(nseed0,size=1)[0])

A       = cs.loadmixmatrix(params_path.pathout,"mixmatrix")
if not os.path.isdir(os.path.join(params_path.pathout,subdirs[0])):
    os.makedirs(os.path.join(params_path.pathout,subdirs[0]))
    
bar = progressbar.ProgressBar(maxval=names.size)
for i,iname in enumerate(names):
    clear_output(wait=True)
    bar.update(i)    
    params_cs, params_wt = cs.load(params_CS,params_WT)
    params_path["name_noise"] = iname
    X = cs.getmaps(params_maps, params_path)
    X = cs.adaptation_maps(X, params_maps, params_path)
    params_maps["iseed"]="L"+str(nseed[i])
    cs.saveouts(mrec=X, A=A[L0], params_path=params_path, params_maps=params_maps, params_WT=params_wt, params_CS=params_cs, subdirs=subdirs)    


bar.finish()

100% (3 of 3) |##########################| Elapsed Time: 0:03:48 Time:  0:03:48


### 21cm  proj prior
$C_{\ell}^{\textrm{proj prior}}$

In [10]:
params_maps["getdata"] = "prior"
subdirs = ["projprior"]
if not os.path.isdir(os.path.join(params_path.pathout,subdirs[0])):
    os.makedirs(os.path.join(params_path.pathout,subdirs[0]))

names, nseed = cs.nsamples(params_CS,params_path,path=params_path.dir_prior, listdir="proj"+params_maps["getdata"],nseed_used=nseed0)


if nseed0[nseed0==params_CS.seed_used].size>0:
    L0 = "L{}".format(params_CS.seed_used)
else:
    L0 = "L{}".format(np.random.choice(nseed0,size=1)[0]) 

A = cs.loadmixmatrix(params_path.pathout,"mixmatrix")

if not os.path.isdir(os.path.join(params_path.pathout,subdirs[0])):
    os.makedirs(os.path.join(params_path.pathout,subdirs[0]))

bar = progressbar.ProgressBar(maxval=names.size)
for i,iname in enumerate(names):
    clear_output(wait=True)
    bar.update(i)    
    params_cs, params_wt = cs.load(params_CS,params_WT)
    params_path["name_prior"] = iname
    X = cs.getmaps(params_maps, params_path)
    X = cs.adaptation_maps(X, params_maps, params_path)
    params_maps["iseed"]="L"+str(nseed[i])
    cs.saveouts(mrec=X, A=A[L0], params_path=params_path, params_maps=params_maps, params_WT=params_wt, params_CS=params_cs, subdirs=subdirs)    

bar.finish()

100% (3 of 3) |##########################| Elapsed Time: 0:03:51 Time:  0:03:51


### Saving used parameters

In [11]:
params_CS["seed_used"]=L0
import warnings
warnings.filterwarnings("ignore")
params_maps.to_csv(os.path.join(params_path.pathout,"map_parameters.txt") , sep=':', index=True)
params_CS.to_csv(  os.path.join(params_path.pathout,"CS_parameters.txt")  , sep=':', index=True)
params_WT.to_csv(  os.path.join(params_path.pathout,"WT_parameters.txt")  , sep=':', index=True)
params_path.to_csv(os.path.join(params_path.pathout,"path_parameters.txt"), sep=':', index=True)